In [0]:
import pandas as pd
import re
import json
import os
import sys
import numpy as np
import pickle

# load functions
with open('project_config.json','r') as fp: 
    project_config = json.load(fp)
 
module_path = os.path.join(project_config['project_module_relative_path'])
sys.path.append(module_path)
 
from data_processing import *

### Read in files

In [0]:
# Create a list for each row
json_data =[]

# Read in json by each lind
with open('./evidence-10-02-23-split.json') as f:
    for line in f:
        json_data.append(json.loads(line))

In [0]:
# Convert to dataframe
df = pd.DataFrame(json_data)

In [0]:
# Unpack the evidence from its json structure, so we get the text and themeIds, issueIds, id, and status all as its own column
evidence_list = []
for index, row in df.iterrows():
    evidence={}
    evidence["id"] = row["_id"]
    evidence["status"]= row["_source"].get("status", "")
    text_translated = row["_source"].get("textTranslated", {})
    evidence["text"] = text_translated.get("en", "")
    evidence["themeIds"] = row["_source"].get("themeIds", np.NaN)
    evidence["issueIds"] = row["_source"].get("issueIds", "")
    evidence_list.append(evidence)

In [0]:
# convert into dataframe
evidence=pd.DataFrame(evidence_list)

In [0]:
evidence.head()

In [0]:
# read in audit data
audit = pd.read_csv("audit_10042023.csv")

In [0]:
audit.head()

In [0]:
# clean audit columns
cols_to_clean = ["themeIdsReviewed", "themeIdsSystem","themeIdsSystemFalseNegatives", "themeIdsSystemFalsePositives"]

for column in cols_to_clean:
    audit = string_to_list_column(audit, column)

In [0]:
# merge with new data
amp = pd.merge(evidence, audit, how = "left", on = "id")

In [0]:
# write to pickle
amp.to_pickle("./model_training_data.pkl")  
audit.to_pickle("./audit_model_training_data.pkl")  